In [1]:
import itertools
import copy
import collections
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import pickle
import random

In [ ]:
######################
### Generate sequences
######################

def generate_trials(operators, input_ids, len_seq, init_values, replacement=True):
    
    ''' This function defines all possible permutations of initial value & sequence of input cues and operators.
    Args: 
        operators, input_ids, init_values: lists of operator, input cue and initial values
        len_seq: number of operation and input pairs per sequence
        replacement: whether an operation/input can be repeated in a sequence
    Returns:
        Output is an array of shape n X len_seq+1.
        Each row is one of n unique ordered permutations.
        First column indicates the initial value at t=0.
        Every following column contains a tuple, where the first position indicates
        the operator and the second position indicates the input cue.
        Final column indicates the outcome of the opperatioons on the initial value'''
    
    seq = []
    if replacement:
        combi_inputcue = list(itertools.product(input_ids, repeat=len_seq))
        combi_operators = list(itertools.product(operators, repeat=len_seq))
    else:
        if len_seq == 2: # if seq of 2 sample binary cue evenly
            combi_inputcue = list(itertools.permutations(input_ids, len_seq))
        else:
            combi_inputcue = list(itertools.product(input_ids, repeat=len_seq))
        combi_operators = list(itertools.permutations(operators, len_seq))

    for init in init_values:
        for cue in combi_inputcue:
            for op in combi_operators:
                #seq.append([(init, np.nan, np.nan),
                seq.append([init,
                            *zip(tuple(op), cue)]) #group per time point t

    return seq

def operate_op(currval, step_tuple, cue_dict):
    """   """
    nextval = cue_dict[step_tuple[1]]
    if step_tuple[0] == '+': # add
        currval = currval + nextval
    elif step_tuple[0] == '*': # multiply
        currval = currval * nextval
    return currval

def calculate_output(step_tuple, cue_dict):
    """ """
    curr_val = step_tuple[0]
    for i in range(1,len(step_tuple)):
        curr_val = operate_op(curr_val, step_tuple[i], cue_dict)
    return curr_val


def generate_sequences(operators, input_ids, len_seq, init_values, cue_dict, replacement=True):
    """ """
    all_trials = generate_trials(operators, input_ids, len_seq, init_values, replacement)
    for trial in all_trials:
        trial_output = calculate_output(trial, cue_dict)
        trial.append(trial_output)
    
    return(all_trials)
   
    
    
##################################################
## Transform data to rnn data - continuous output
##################################################

class SequenceData(Dataset):
    def __init__(self, data, labels, seq_len, stages, cont_out):
        'Initialization'
        self.data = convert_seq2onehot(data, stages)
        self.seq_len = seq_len
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sequence = self.data[index,:].astype(np.float32)
        out_state = np.array(self.labels[index]).astype(np.float32)
        return sequence, out_state
    
    
def convert_seq2inputs(sequences, num_classes=13, seq_len=5, stages = False, cont_out = True ):
    '''
    converts sequences as they are generated by generate_experiment_lists.py
    into input to be fed into RNN (one-hote encoded)
    Parameters:
        sequences: list of trials with format : [initial_value, (operation, input_cue) ... , output_value]
        num_classes: total number of features for onehot encoding
        seq_len: number of time steps per sequence
        stages: if False each unit is a time step, if True each tuple is a time step
        cont_out: if True the output is continuous, if False output is categorical
    ''' 
    seq = [sublist[:-1] for sublist in sequences]
    out = [sublist[-1] for sublist in sequences]
    
    seqdata = SequenceData(seq, out, seq_len, stages, cont_out)

    return seqdata


def convert_seq2onehot(seq, num_classes=13, stages):
    """
    """
    data = []
    for trial in seq:
        trial_data = []
        for i,t in enumerate(trial):
            if i==0:
                init = torch.zeros(num_classes)
                init[0] = t
                trial_data.append(init)
                continue
            else:
                op = torch.tensor(convert_operation[t[0]])
                op = torch.nn.functional.one_hot(op, num_classes=num_classes)
                inputcue = torch.tensor(convert_inputcue[t[1]])
                inputcue = torch.nn.functional.one_hot(inputcue, num_classes=num_classes)
                if stages:
                    op_cue = op + inputcue
                    trial_data.append(op_cue)
                else:
                    trial_data.append(op)
                    trial_data.append(inputcue)
        data.append(torch.stack(trial_data))
        
    data = torch.stack(data,dim=0) #combine into tensor of shape n_trials X n_time_steps X inputvector_size
    data = data.numpy()
    
    return data




